In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime

from pathlib import Path

from collections import Counter
from tqdm.cli import tqdm

In [3]:


data_folder = Path("data")
figs_folder = Path("figs")

df = df = pd.read_csv(
    data_folder / "Apple Music Play Activity.csv", 
    parse_dates=[
        'Event End Timestamp', 
        'Event Received Timestamp',
        'Event Start Timestamp'
    ]
)
df["full_comp_name"] = df.apply(lambda r: f"{r['Artist Name']} - {r['Content Name']}", axis=1)


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="861ff78243574150bdd2e1f8ab730e69",
                                                           client_secret="585bc7c087e6446eb3cb607c7a85e591"))

spotify_df = pd.DataFrame()

for request_name in tqdm(df["full_comp_name"].unique()):
    results = sp.search(q=request_name, limit=1)
    if len(results['tracks']['items']) > 0:
        spotify_df = spotify_df.append({
            "full_comp_name": request_name,
            "search_responce": json.dumps(results['tracks']['items'][0]),
        }, ignore_index=True)
        


track_ids = pd.DataFrame.from_dict(spotify_df.search_responce.apply(lambda s: json.loads(s)).tolist())["id"].tolist()

def track_ids_generator():
    for i in range(0, len(track_ids), 100):
        yield track_ids[i:i + 100]

results = []
for track_ids_batch in tqdm(track_ids_generator(), total=len(track_ids) / 100):
    results.extend(sp.audio_features(tracks=track_ids_batch))


spotify_df = pd.concat((
    spotify_df.full_comp_name,
    pd.DataFrame.from_dict(spotify_df.search_responce.apply(lambda s: json.loads(s)).tolist()),
), axis=1)

spotify_df = pd.merge(
    pd.DataFrame.from_dict(spotify_df), 
    pd.DataFrame.from_dict(results),
    on="id"
)

100%|██████████| 3791/3791 [08:51<00:00,  7.13it/s]
 98%|█████████▊| 31/31.7 [00:08<00:00,  3.73it/s]/home/iovcharenko/miniconda3/envs/data-viz/lib/python3.8/site-packages/tqdm/std.py:516: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(
101%|██████████| 32/31.7 [00:08<00:00,  3.89it/s]


In [7]:
spotify_df.columns

Index(['full_comp_name', 'album', 'artists', 'available_markets',
       'disc_number', 'duration_ms_x', 'explicit', 'external_ids',
       'external_urls', 'href', 'id', 'is_local', 'name', 'popularity',
       'preview_url', 'track_number', 'type_x', 'uri_x', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type_y', 'uri_y',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature'],
      dtype='object')

In [26]:
# json columns
spotify_df[["album", "artists", "available_markets", "external_ids", "external_urls"]]

,album,artists,available_markets,external_ids,external_urls
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'QMMZN1801602'},{'spotify': 'https://open.spotify.com/track/3O...
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'USSM16300086'},{'spotify': 'https://open.spotify.com/track/5h...
2,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'USSM10004123'},{'spotify': 'https://open.spotify.com/track/2B...
3,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'USMTD1101887'},{'spotify': 'https://open.spotify.com/track/0J...
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'USA2B1302335'},{'spotify': 'https://open.spotify.com/track/0R...
...,...,...,...,...,...
3263,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AR, BO, BR, CA, CL, CO, CR, EC, GT, HN, MX, N...",{'isrc': 'USVI29101019'},{'spotify': 'https://open.spotify.com/track/1W...
3264,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'US3KM1104007'},{'spotify': 'https://open.spotify.com/track/68...
3265,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'USEE16900011'},{'spotify': 'https://open.spotify.com/track/4q...
3266,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",{'isrc': 'GBCFB1900100'},{'spotify': 'https://open.spotify.com/track/3r...


In [27]:
for col in ["album", "artists", "available_markets", "external_ids", "external_urls"]:
    spotify_df[col] = spotify_df[col].apply(lambda obj: json.dumps(obj))

In [28]:
spotify_df.to_csv(data_folder / "_spotify_v3.csv", index=False)

# Add album cover

In [ ]:
# import cv2
# from colormap import rgb2hex
# import requests

# from PIL import Image


# def get_mean_color(cover_url):
#     none_color = "#e0e0e0"
    
#     if cover_url is None:
#         return none_color
    
#     try:
#         cover_img = Image.open(requests.get(cover_url, stream=True).raw)
#         rgb_mean = np.transpose(np.array(cover_img), (2, 0, 1)) \
#                       .reshape((3, -1)) \
#                       .mean(1).astype(np.uint8)
    
#     except Exception as e:
#         print(cover_url)
#         print(e)
#         return none_color
#     return rgb2hex(*rgb_mean)


# tqdm.pandas()
# album_df["album_cover_color"] = album_df["album_cover_url"].progress_apply(get_mean_color)